In [13]:
# Import packages
import yfinance as yf
import requests

from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager

### Strategy

The stock screening strategy below is designed to complement the VWAP and ABCD pattern day trading strategies outlined in the book "Day Trading for a Living" by

#### Pre-Market Gappers
Criteria for the Gappers Scanner includes:
- Stocks that in the pre-market gapped up or down at least 2%
- Stocks that have traded at least 100,000 shares by 9am in pre-market
- Stocks that have an average daily volume of over 500,000 shares
- Stocks that have Average True Range (ATR) of at least 50 cents
- There is a fundamental catalyst for the stock

#### Additional Filter
We will filter for only Medium Float stocks (20 to 500 million shares) with a price range between $10-$100 and Large float stocks (500+ million shares) with a price range > $20. These stock work best with the VWAP and ABCD pattern trading strategies.

#### Human Screen
I will review the output manually and do my own research into whether there is a fundamental catalyst for the stock


In [14]:
# Lets begin by webscraping a list of the top 100 losers and top 100 gainers for the day

# Gainers
t100_gainers_url = "https://finance.yahoo.com/screener/predefined/day_gainers?count=100&offset=0"

# Losers
t100_losers_url = "https://finance.yahoo.com/screener/predefined/day_losers?count=100&offset=0"

def scrape_gappers(url, driver):
    driver.get(url)
    print(driver.current_url)
    


In [15]:
options = webdriver.ChromeOptions() 
options.headless = True 
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install())) 

scrape_gappers(t100_gainers_url, driver)

[WDM] - Downloading: 100%|██████████| 6.80M/6.80M [00:00<00:00, 11.3MB/s]


WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0025DCE3+50899]
	(No symbol) [0x001EE111]
	(No symbol) [0x000F5588]
	(No symbol) [0x00110AAB]
	(No symbol) [0x0010F479]
	(No symbol) [0x00141FFE]
	(No symbol) [0x00141CEC]
	(No symbol) [0x0013B6F6]
	(No symbol) [0x00117708]
	(No symbol) [0x0011886D]
	GetHandleVerifier [0x004C3EAE+2566302]
	GetHandleVerifier [0x004F92B1+2784417]
	GetHandleVerifier [0x004F327C+2759788]
	GetHandleVerifier [0x002F5740+672048]
	(No symbol) [0x001F8872]
	(No symbol) [0x001F41C8]
	(No symbol) [0x001F42AB]
	(No symbol) [0x001E71B7]
	BaseThreadInitThunk [0x75CB00F9+25]
	RtlGetAppContainerNamedObjectPath [0x77E27BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77E27B8E+238]
	(No symbol) [0x00000000]
